In [68]:
from typing import List, Self
import pydantic, os, dotenv, PyPDF2, chromadb, chromadb.config, chromadb.utils.embedding_functions
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

dotenv.load_dotenv()

True

In [14]:
ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20", temperature=0.7).invoke("Hey")

AIMessage(content='Hey there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash-latest', 'safety_ratings': []}, id='run--82b2aa09-71a7-4a07-8f4e-e8cd7737bcf3-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [ ]:
emb = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004").embed_query("cats")
len(emb), emb[:2]  # Size 768

(768, [-0.03150507062673569, -0.001939397887326777])

In [48]:
chroma_compatible_google_ef = chromadb.utils.embedding_functions.ChromaLangchainEmbeddingFunction(
    embedding_function = GoogleGenerativeAIEmbeddings(
        model = "models/text-embedding-004"
    )
)

client = chromadb.PersistentClient(path=os.path.join(os.getcwd(), "database", "chroma_db"))

documents = client.get_or_create_collection(
    name="documents",
    embedding_function=chroma_compatible_google_ef
)
titles = client.get_or_create_collection(
    name="titles",
    embedding_function=chroma_compatible_google_ef
)

In [44]:
def load_documents() -> List[str]:
    documents = []
    documents_dir = os.path.join(os.getcwd(), "database", "documents")
    for filename in os.listdir(documents_dir):
        if filename.endswith(".txt") or filename.endswith(".md") or filename.endswith(".html"):
            with open(os.path.join(documents_dir, filename), "r", encoding="utf-8") as file:
                documents.append(file.read())
        elif filename.endswith(".pdf"):
            with open(os.path.join(documents_dir, filename), "rb") as file:
                reader = PyPDF2.PdfReader(file)
                text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
                documents.append(text)
    return documents

In [58]:
if True:
    documents.delete([f"doc_{i+1}" for i in range(documents.count())])
    titles.delete([f"doc_{i+1}" for i in range(titles.count())])

In [ ]:
if True:
    docs = load_documents()
    
    generate_title = lambda doc: ChatGoogleGenerativeAI(
            model="gemini-2.5-flash-preview-05-20",
            temperature=0.7
        ).invoke(
            "Generate a title for the following document in the language of said document. "
            + f"Only output the title and nothing else. Document:\n\n{doc}"
        ).content
    
    documents.add(documents=docs, ids=[f"doc_{i+1}" for i in range(len(docs))])
    titles.add(documents=[generate_title(doc) for doc in docs], ids=[f"doc_{i+1}" for i in range(len(docs))])
    
documents.count()

9

In [71]:
documents.query(
    query_texts=["chats"],
    n_results=3,
)

{'ids': [['doc_2', 'doc_4', 'doc_5']],
 'embeddings': None,
 'documents': [[' \nDéroulement\n \nTX\n \n \nPhase\n \nExploratoire\n \n \nPendant\n \nles\n \ndeux\n \npremières\n \nsemaines,\n \nnous\n \nétudions\n \nles\n \ndifférents\n \nservices\n \npossibles\n \nqui\n \npourraient\n \npermettre\n \nde\n \ntirer\n \npartie\n \ndes\n \nmodèles\n \nde\n \nlangage\n \npour\n \nles\n \nactivités\n \nquotidiennes\n \nà\n \nl’UTC.\n \n \nOn\n \nréfléchit\n \naux\n \ndifférentes\n \nproblématiques\n \nauxquelles\n \nla\n \ntechnologie\n \nse\n \nprête\n \net\n \non\n \névalue\n \nen\n \ndirect\n \nla\n \nfaisabilité\n \nde\n \nnos\n \nidées,\n \nalors\n \nque\n \nnous\n \ndécouvrons\n \nles\n \ntechnologies\n \ndisponibles.\n \n \nOn\n \ns’arrête\n \nnotamment\n \nsur\n \nl’idée\n \nde\n \ncréer\n \nun\n \nagent\n \nqui\n \npourrait\n \ntirer\n \npartie\n \nde\n \nla\n \ndocumentation\n \nde\n \nla\n \nDSI\n \npour\n \nassister\n \nles\n \nutilisateurs.\n \n \nEtablissement\n \ndes\n \nContr

In [62]:
class Document(pydantic.BaseModel):
    id: str
    title: str
    content: str

def query(q: str, n_results: int) -> List[Document]:
    document_results = documents.query(
        query_texts=[q],
        n_results=n_results,
    )
    if not document_results:
        return []
    
    ids = document_results["ids"][0]
    titles_results = titles.get(ids=ids)
    return [
        Document(
            id=ids[i],
            title=titles_results["documents"][i],
            content=document_results["documents"][0][i]
        ) for i in range(len(ids))
    ]
    
query("Anything about cats", 3)

[Document(id='doc_3', title='Development of a Conversational Agent for IT Support at UTC using Large Language Models', content="Chat\nNom vulgaire ou nom vernaculaire ambigu :\nl'appellation « Chat » s'applique en français à plusieurs\ntaxons distincts.\nUn Chat de Biet, (Felis bieti)\nTaxons concernés\nDans la famille des Felidae , certaines\nespèces dans les genres:\nFelis\nLeopardus\nOtocolobus\nPardofelis\nPrionailurus\nChat (animal)\nChat  est un terme ambigu employé en français\npour désigner de nombreux félins  de taille\nmoyenne ou petite, appartenant à la sous-famille\ndes félinés . Employé seul, il s'agit du nom\nvernaculaire  donné au chat domestique  (Felis\nsilvestris catus ). Par synecdoque , chat peut\ndésigner l'ensemble des félins  : l'expression « chat\nsauvage  » s'applique à plusieurs espèces ou sous-\nespèces de petits félins, tandis que la sous-famille\ndes Panthérinés  est parfois désignée par\nl'expression « grands chats » .\nListe alphabétique de noms vulgaires

In [67]:
def LLMSummary(q: str) -> str:
    return ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-preview-05-20",
        temperature=0.7
    ).invoke(
        "Summarize the following documents in a single markdown setup. Use markdown freely. "
        + "Use the language of the user query (usually french), not the language of the documents. "
        + "Only output the summary and nothing else. "
        + f"\nOriginal user query: {q}"
        + "\nDocuments:\n"
        + "\n".join([f"{doc.title}\n{doc.content}" for doc in query(q, 3)])
    ).content
    
LLMSummary("Je cherche des docs qui parlent de linguistique")

'Ces documents couvrent deux sujets distincts liés à la linguistique.  Le premier est une exploration philosophique et pragmatique de notre expérience subjective, de notre rapport au monde et à la communication. Il met l\'accent sur le rôle primordial de l\'expérience vécue dans la construction de notre compréhension du monde, l\'anticipation du futur et l\'interaction avec les autres.  L\'analyse critique du langage, notamment l\'identification des "boucles langagières" (phrases toutes faites déconnectées de l\'expérience) et la promotion d\'un langage plus concret et expérientiel, sont au cœur de cette approche. L\'objectif est de développer une conscience accrue de nos processus mentaux pour mieux s\'aligner sur ses aspirations profondes.\n\nLe second document est une étude linguistique formelle du Khoekhoe, une langue Khoisan.  Il explore la notion d\'« omniprédicativité », où tous les mots peuvent fonctionner comme prédicats. L\'analyse se concentre sur la structure des groupes no

In [74]:
def stream_summary(q: str):
    for tok in ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-preview-05-20",
        temperature=0.7
    ).stream(
        "Summarize the following documents in a single markdown setup. Use markdown freely. "
        + "Use the language of the user query (usually french), not the language of the documents. "
        + "Only output the summary and nothing else. "
        + f"\nOriginal user query: {q}"
        + "\nDocuments:\n"
        + "\n".join([f"{doc.title}\n{doc.content}" for doc in query(q, 3)])
    ):
        yield tok.content
    
for tok in stream_summary("Je cherche des docs qui parlent de linguistique"):
    print(tok, end="", flush=True)

Ce document résume deux textes. Le premier présente une approche phénoménologique et pragmatique de notre rapport au monde, axée sur l'expérience subjective comme matériau premier de la compréhension, de la décision et de l'interaction. Il souligne l'importance de la conscience des mécanismes d'interprétation du passé, d'anticipation du futur et de communication, afin d'identifier les pièges du langage et de la pensée automatique ("boucles langagières") pour mieux s'aligner sur ses aspirations profondes.  Le second texte est un article académique analysant la structure des phrases nominales et la prédication dans la langue Khoekhoe.  Il explore le concept de "langue omniprédicative", où tous les mots peuvent fonctionner comme prédicats. L'analyse propose une approche formelle en HPSG, montrant que les syntagmes nominaux sont dérivés de manière uniforme comme projections d'éléments pronominaux, modifiés par des propositions relatives.  L'article détaille la structure grammaticale de la 